In [1]:
import numpy as np
import pandas as pd
import datetime
import category_encoders as ce
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 총 구매 횟수
def pre_buy_num():
    data01 = pd.read_csv('data/LPOINT_BIG_COMP_01_DEMO.csv')
    data02 = pd.read_csv('data/LPOINT_BIG_COMP_02_PDDE.csv')
    data03 = pd.read_csv('data/LPOINT_BIG_COMP_03_COP_U.csv')
    data04 = pd.read_csv('data/LPOINT_BIG_COMP_04_PD_CLAC.csv')
    data05 = pd.read_csv('data/LPOINT_BIG_COMP_05_BR.csv')
    data06 = pd.read_csv('data/LPOINT_BIG_COMP_06_LPAY.csv')

    data01.columns = ['고객번호', '성별', '연령대', '거주지대분류코드']
    data02.columns = ['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매시간', '구매금액', '구매수량']
    data03.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '이용일자', '방문일자', '이용시간', '이용금액']
    data04.columns = ['상품코드', '소분류명', '대분류명', '중분류명']
    data05.columns = ['점포코드', '제휴사', '점포대분류코드', '점포중분류코드']
    data06.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '이용일자', '이용시간', '이용금액']

    # 성별 One-Hot encoding
    data01 = pd.get_dummies(data=data01,columns=['성별'])
    data01 = data01.drop('성별_남성', axis = 1)

    # 연령대 Label encoding
    age = {'20대':1, '30대':2, '40대':3, '50대':4, '60대':5, '70대':6}
    data01['연령대'] = data01['연령대'].replace(age)

    # 거주지대분류코드 Binary encoding
    data01 = ce.BinaryEncoder(cols = ['거주지대분류코드']).fit_transform(data01)

    # L-Point 이용횟수, 평균 이용금액 계산
    LP_num = data06.groupby(['고객번호','이용일자']).size().groupby(['고객번호']).size().rename('LP이용횟수')
    LP_amount = data06[['고객번호','이용금액']].groupby(['고객번호']).mean().rename(columns = {'이용금액' : '평균LP이용금액'})

    # 구매일자 기준 고객번호, 상품코드별 총 구매 횟수 추가, 고객번호 기준 L-Point 이용횟수, 평균 이용금액 병합
    data02 = (data02.merge(data02.groupby(['고객번호', '상품코드', '구매일자']).size().groupby(['고객번호', '상품코드']).size().rename("총구매횟수").reset_index())
            .merge(LP_num, how='left', left_on='고객번호', right_on='고객번호')
            .merge(LP_amount, how='left', left_on='고객번호', right_on='고객번호'))
    data02['LP이용횟수'] = data02['LP이용횟수'].fillna(0)
    data02['평균LP이용금액'] = data02['평균LP이용금액'].fillna(0)

    # 구매일자 datetime 타입으로 변환, 구매요일 추가
    buy_date = pd.to_datetime(data02['구매일자'], format="%Y%m%d")
    data02['구매요일']=buy_date.dt.dayofweek
    data02['구매일자']=buy_date.dt.dayofyear
    data02 = data02[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매요일', '구매시간', '구매금액','구매수량', '총구매횟수', 'LP이용횟수', '평균LP이용금액']]

    # 구매금액, 평균LP이용금액 Log 변환
    data02['구매금액']=np.log1p(data02['구매금액'])
    data02['평균LP이용금액']=np.log1p(data02['평균LP이용금액'])

    # 제휴사 Binary encoding, data01, data04와 병합
    data02 = ce.BinaryEncoder(cols = ['제휴사']).fit_transform(data02)
    data02 = data02.merge(data01).merge(data04)

    # cardinality 높은 범주형 변수 Mean encoding, 고객번호, 영수증번호 삭제
    target = '총구매횟수'
    high_card = ['상품코드', '중분류명', '대분류명']
    for i in high_card:
        data02[f'{i}_mean'] = data02[i].map(data02.groupby(i)[target].mean())
    data02.drop(['고객번호', '영수증번호', '상품코드', '소분류명', '중분류명', '대분류명'], axis = 1, inplace = True)

    # 채널구분 기준 offline 데이터, online 데이터 분리, offline 데이터는 data05와 병합
    data02_offline = data02[data02['채널구분'] == 1].join(data05.set_index('점포코드')[['점포중분류코드', '점포대분류코드']], on='점포코드').drop('채널구분', axis = 1)
    data02_online = data02[data02['채널구분'] == 2].drop('점포코드', axis = 1).drop('채널구분', axis = 1)

    # data02_offline 점포코드 관련 변수 Mean encoding
    high_card = ['점포코드', '점포중분류코드', '점포대분류코드']
    for i in high_card:
        data02_offline[f'{i}_mean'] = data02_offline[i].map(data02_offline.groupby(i)[target].mean())
    data02_offline.drop(['점포코드', '점포중분류코드', '점포대분류코드'], axis = 1, inplace = True)

    # 결측치 삭제
    data03.dropna(axis=0, inplace=True)

    # 이용일자 기준 고객번호, 점포코드별 총 이용 횟수 추가, 고객번호 기준 L-Point 이용횟수, 평균 이용금액 병합
    data03 = (data03.merge(data03.groupby(['고객번호', '점포코드', '이용일자']).size().groupby(['고객번호', '점포코드']).size().rename("총이용횟수").reset_index())
            .merge(LP_num, how='left', left_on='고객번호', right_on='고객번호')
            .merge(LP_amount, how='left', left_on='고객번호', right_on='고객번호'))
    data03['LP이용횟수'] = data03['LP이용횟수'].fillna(0)
    data03['평균LP이용금액'] = data03['평균LP이용금액'].fillna(0)

    # 이용일자 datetime 타입으로 변환, 이용요일 추가, 방문일자 삭제
    buy_date = pd.to_datetime(data03['이용일자'], format="%Y%m%d")
    data03['이용요일']=buy_date.dt.dayofweek
    data03['이용일자']=buy_date.dt.dayofyear
    data03.drop('방문일자', axis = 1, inplace = True)
    data03 = data03[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '이용일자', '이용요일', '이용시간', '이용금액', '총이용횟수', 'LP이용횟수', '평균LP이용금액']]

    # 이용금액, 평균LP이용금액 Log 변환
    data03['이용금액']=np.log1p(data03['이용금액'])
    data03['평균LP이용금액']=np.log1p(data03['평균LP이용금액'])

    # 제휴사 Binary Encoding, data01, 05와 병합
    data03 = ce.BinaryEncoder(cols = ['제휴사']).fit_transform(data03)
    data03 = data03.merge(data01).join(data05.set_index('점포코드')[['점포중분류코드', '점포대분류코드']], on='점포코드')

    # cardinality 높은 범주형 변수 Mean encoding, 고객번호, 영수증번호 삭제
    target = '총이용횟수'
    high_card = ['점포코드', '점포중분류코드', '점포대분류코드']
    for i in high_card:
        data03[f'{i}_mean'] = data03[i].map(data03.groupby(i)[target].mean())
    data03.drop(['고객번호', '영수증번호', '점포코드', '점포중분류코드', '점포대분류코드'], axis = 1, inplace = True)

    # 채널구분 기준 offline 데이터, online 데이터 분리
    data03_offline = data03[data03['채널구분'] == 1].drop('채널구분', axis = 1)
    data03_online = data03[data03['채널구분'] == 2].drop('채널구분', axis = 1)

    return data02_offline, data02_online, data03_offline, data03_online

In [3]:
def pre_buy_amount_round():
    data01 = pd.read_csv('data/LPOINT_BIG_COMP_01_DEMO.csv')
    data02 = pd.read_csv('data/LPOINT_BIG_COMP_02_PDDE.csv')
    data03 = pd.read_csv('data/LPOINT_BIG_COMP_03_COP_U.csv')
    data04 = pd.read_csv('data/LPOINT_BIG_COMP_04_PD_CLAC.csv')
    data05 = pd.read_csv('data/LPOINT_BIG_COMP_05_BR.csv')
    data06 = pd.read_csv('data/LPOINT_BIG_COMP_06_LPAY.csv')

    data01.columns = ['고객번호', '성별', '연령대', '거주지대분류코드']
    data02.columns = ['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매시간', '구매금액', '구매수량']
    data03.columns = ['고객번호', '영수증번호', '제휴사', '점포코드', '채널구분', '이용일자', '방문일자', '이용시간', '이용금액']
    data04.columns = ['상품코드', '소분류명', '대분류명', '중분류명']
    data05.columns = ['점포코드', '제휴사', '점포대분류코드', '점포중분류코드']
    data06.columns = ['고객번호', '영수증번호', '제휴사', '채널구분', '이용일자', '이용시간', '이용금액']

    # 성별 One-Hot encoding
    data01 = pd.get_dummies(data=data01,columns=['성별'])
    data01 = data01.drop('성별_남성', axis = 1)

    # 연령대 Label encoding
    age = {'20대':1, '30대':2, '40대':3, '50대':4, '60대':5, '70대':6}
    data01['연령대'] = data01['연령대'].replace(age)

    # 거주지대분류코드 Binary encoding
    data01 = ce.BinaryEncoder(cols = ['거주지대분류코드']).fit_transform(data01)

    # L-Point 이용횟수, 평균 이용금액 계산
    LP_num = data06.groupby(['고객번호','이용일자']).size().groupby(['고객번호']).size().rename('LP이용횟수')
    LP_amount = data06[['고객번호','이용금액']].groupby(['고객번호']).mean().rename(columns = {'이용금액' : '평균LP이용금액'})

    # 구매일자 datetime 타입으로 변환, 구매요일, 구매월 추가
    buy_date = pd.to_datetime(data02['구매일자'], format="%Y%m%d")
    data02['구매일자']=buy_date
    data02['구매요일']=buy_date.dt.dayofweek #월:0, 화:1, 수:2, 목:3, 금:4, 토:5, 일:6
    data02['구매월']=buy_date.dt.month

    # 구매일자 기준 고객번호, 상품코드별 총 구매 횟수
    data02_1 = data02.groupby(['고객번호', '상품코드', '구매일자']).size().groupby(['고객번호', '상품코드']).size().rename("총구매횟수").reset_index()

    # 고객번호, 상품코드별 총 구매 금액, 천원 단위 반올림
    data02_2 = round(data02.groupby(['고객번호', '상품코드']).sum()['구매금액'].rename("총구매금액"), -3).reset_index()

    # 구매일자 기준 고객번호, 상품코드별 구매 횟수(순차적 증가)
    data02_3 = data02[['고객번호','상품코드','구매일자']].groupby(['고객번호', '상품코드']).rank(method='dense').rename(columns = {'구매일자' : '일자별구매횟수'})
    data02 = pd.concat([data02, data02_3],axis=1)

    # 고객번호, 상품코드, 구매월별 평균 구매 금액, 천원 단위 반올림
    data02_4 = round(data02[['고객번호','상품코드','구매월','구매금액']].groupby(['고객번호', '상품코드','구매월']).mean().rename(columns={'구매금액':'월평균구매금액'}),-3).reset_index()

    # 총구매횟수, 총구매금액, L-Point 이용횟수, 평균이용금액 추가
    data02 = data02.merge(data02_1).merge(data02_2).merge(data02_4).merge(LP_num, how='left', left_on='고객번호', right_on='고객번호').merge(LP_amount, how='left', left_on='고객번호', right_on='고객번호')
    data02['LP이용횟수'] = data02['LP이용횟수'].fillna(0)
    data02['평균LP이용금액'] = data02['평균LP이용금액'].fillna(0)
    data02 = data02[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매월', '구매요일', '구매시간', '구매금액','구매수량', '일자별구매횟수', '총구매횟수', '총구매금액', '월평균구매금액','LP이용횟수', '평균LP이용금액']]

    # 제휴사 Binary encoding, data01, data04와 병합
    data02 = ce.BinaryEncoder(cols = ['제휴사']).fit_transform(data02)
    data02 = data02.merge(data01).merge(data04)

    #총구매금액, 월평균구매금액 10000으로 나누기
    data02['총구매금액'] = data02['총구매금액']/10000
    data02['월평균구매금액'] = data02['월평균구매금액']/10000

    # 구매금액, 총구매금액, 월평균구매금액, 평균LP이용금액 Log 변환
    data02['구매금액']=np.log1p(data02['구매금액'])
    data02['총구매금액']=np.log1p(data02['총구매금액'])
    data02['월평균구매금액']=np.log1p(data02['월평균구매금액'])
    data02['평균LP이용금액']=np.log1p(data02['평균LP이용금액'])

    # cardinality 높은 범주형 변수 Mean encoding, 고객번호, 영수증번호, 구매월 삭제
    target = '총구매금액'
    high_card = ['상품코드', '중분류명', '대분류명']
    for i in high_card:
        data02[f'{i}_mean'] = data02[i].map(data02.groupby(i)[target].mean())
    data02.drop(['고객번호', '영수증번호', '구매월','상품코드', '소분류명', '중분류명', '대분류명'], axis = 1, inplace = True)

    #구매일자 1~365범위로 변환
    data02['구매일자']=data02['구매일자'].dt.dayofyear

    # 채널구분 기준 offline 데이터, online 데이터 분리, offline 데이터는 data05와 병합
    data02_offline = data02[data02['채널구분'] == 1].join(data05.set_index('점포코드')[['점포중분류코드', '점포대분류코드']], on='점포코드').drop('채널구분', axis = 1)
    data02_online = data02[data02['채널구분'] == 2].drop('점포코드', axis = 1).drop('채널구분', axis = 1)

    # data02_offline 점포코드 관련 변수 Mean encoding
    high_card = ['점포코드', '점포중분류코드', '점포대분류코드']
    for i in high_card:
        data02_offline[f'{i}_mean'] = data02_offline[i].map(data02_offline.groupby(i)[target].mean())
    data02_offline.drop(['점포코드', '점포중분류코드', '점포대분류코드'], axis = 1, inplace = True)

    # 결측치 삭제
    data03.dropna(axis=0, inplace=True)

    # 이용일자 datetime 타입으로 변환, 이용요일, 이용월 추가, 방문일자 삭제
    buy_date = pd.to_datetime(data03['이용일자'], format="%Y%m%d")
    data03['이용일자']=buy_date
    data03['이용요일']=buy_date.dt.dayofweek
    data03['이용월']=buy_date.dt.month
    data03.drop('방문일자', axis = 1, inplace = True)

    # 이용일자 기준 고객번호, 점포코드별 총 이용 횟수
    data03_1 = data03.groupby(['고객번호', '점포코드', '이용일자']).size().groupby(['고객번호', '점포코드']).size().rename("총이용횟수").reset_index()

    # 고객번호, 점포코드별 총 이용 금액, 천원 단위 반올림
    data03_2 = round(data03.groupby(['고객번호', '점포코드']).sum()['이용금액'].rename("총이용금액"), -3).reset_index()

    # 이용일자 기준 고객번호, 점포코드별 이용 횟수(순차적 증가)
    data03_3 = data03[['고객번호','점포코드','이용일자']].groupby(['고객번호', '점포코드']).rank(method='dense').rename(columns = {'이용일자' : '일자별이용횟수'})
    data03 = pd.concat([data03, data03_3],axis=1)

    # 고객번호, 점포코드, 이용월별 평균 이용 금액, 천원 단위 반올림
    data03_4 = round(data03[['고객번호','점포코드','이용월','이용금액']].groupby(['고객번호', '점포코드','이용월']).mean().rename(columns={'이용금액':'월평균이용금액'}),-3).reset_index()

    # data03에 총이용횟수, 총이용금액, L-Point 이용횟수, 평균이용금액 추가
    data03 = data03.merge(data03_1).merge(data03_2).merge(data03_4).merge(LP_num, how='left', left_on='고객번호', right_on='고객번호').merge(LP_amount, how='left', left_on='고객번호', right_on='고객번호')
    data03['LP이용횟수'] = data03['LP이용횟수'].fillna(0)
    data03['평균LP이용금액'] = data03['평균LP이용금액'].fillna(0)
    data03 = data03[['고객번호', '영수증번호', '채널구분', '제휴사', '점포코드','이용일자', '이용월', '이용요일', '이용시간', '이용금액', '일자별이용횟수', '총이용횟수', '총이용금액', '월평균이용금액','LP이용횟수', '평균LP이용금액']]

    # 제휴사 Binary Encoding, data01, 05와 병합
    data03 = ce.BinaryEncoder(cols = ['제휴사']).fit_transform(data03)
    data03 = data03.merge(data01).join(data05.set_index('점포코드')[['점포중분류코드', '점포대분류코드']], on='점포코드')

    #총이용금액, 월평균이용금액 10000 나누기
    data03['총이용금액'] = data03['총이용금액']/10000
    data03['월평균이용금액'] = data03['월평균이용금액']/10000

    # 이용금액, 총이용금액, 월평균이용금액, 평균LP이용금액 Log 변환
    data03['이용금액']=np.log1p(data03['이용금액'])
    data03['총이용금액']=np.log1p(data03['총이용금액'])
    data03['월평균이용금액']=np.log1p(data03['월평균이용금액'])
    data03['평균LP이용금액']=np.log1p(data03['평균LP이용금액'])

    # cardinality 높은 범주형 변수 Mean encoding, 고객번호, 영수증번호, 이용월 삭제
    target = '총이용금액'
    high_card = ['점포코드', '점포중분류코드', '점포대분류코드']
    for i in high_card:
        data03[f'{i}_mean'] = data03[i].map(data03.groupby(i)[target].mean())
    data03.drop(['고객번호', '영수증번호', '이용월', '점포코드', '점포중분류코드', '점포대분류코드'], axis = 1, inplace = True)

    #이용일자 1~365범위로 변환
    data03['이용일자']=data03['이용일자'].dt.dayofyear

    # 채널구분 기준 offline 데이터, online 데이터 분리
    data03_offline = data03[data03['채널구분'] == 1].drop('채널구분', axis = 1)
    data03_online = data03[data03['채널구분'] == 2].drop('채널구분', axis = 1)

    return data02_offline, data02_online, data03_offline, data03_online

In [4]:
# 데이터에 따른 target 변수 설정, train, test data 분리
def data_split(data):
    if '총구매금액' in data.columns:
        target = data['총구매금액']
        data = data.drop('총구매금액', axis = 1)
    elif '총이용금액' in data.columns:
        target = data['총이용금액']
        data = data.drop('총이용금액', axis = 1)
    elif '총구매횟수' in data.columns:
        target = data['총구매횟수']
        data = data.drop('총구매횟수', axis = 1)
    else:
        target = data['총이용횟수']
        data = data.drop('총이용횟수', axis = 1)
    return train_test_split(data, target, test_size=0.3, random_state=2022)